# Visualization Experiments

Lets load the data artefacts to local memory. These files are to be downloaded from S3 as the pipeline automatically uploads them to the pre-configured S3 bucket.

In [ ]:
from file_util import download_files
import pickle

# Download files from S3 bucket. You can download multiple files at a time by passing a list of names
files_to_download = ["df.pkl", "mapping.pkl"]


In [ ]:
# Download spacy model for the first time
!spacy download en_core_web_md


In [16]:
import spacy

spaCy_model = "en_core_web_md"
nlp = spacy.load(spaCy_model)
stopwords = nlp.Defaults.stop_words


## Scatter plot of transcription with Topic modelling

Change the values of "category", "category_name" to one agenda topic and change the value of "not_category_name" and see different plots.

In [17]:
import pandas as pd
import scattertext as st


def plot_topic_modelling_and_word_to_sentence_search(df, cat_1, cat_1_name, cat_2_name):
    df = df.assign(parse=lambda df: df.text.apply(st.whitespace_nlp_with_sentences))

    corpus = st.CorpusFromParsedDocuments(
        df, category_col='ts_to_topic_mapping_top_1', parsed_col='parse'
    ).build().get_unigram_corpus().remove_terms(stopwords, ignore_absences=True).compact(st.AssociationCompactor(2000))
    
    html = st.produce_scattertext_explorer(
        corpus,
        category=cat_1, category_name=cat_1_name, not_category_name=cat_2_name,
        minimum_term_frequency=0, pmi_threshold_coefficient=0,
        width_in_pixels=1000,
        transform=st.Scalers.dense_rank
    )
    open('./demo_compact.html', 'w').write(html)

plot_topic_modelling_and_word_to_sentence_search(df,
                                                cat_1="TAM",
                                                cat_1_name="TAM",
                                                cat_2_name="Churn")

# once you are done, check the generated HTML file


## Timeline visualizer